# Relevant Resources

CLEAN THIS AND AND INTEGRATE WITH MAIN TOPIC MODELING INTRO SCRIPT TO MAKE A LONGER ONE, THEN TURNT O WORD2VEC


https://nbviewer.jupyter.org/github/bmabey/hacker_news_topic_modelling/blob/master/HN%20Topic%20Model%20Talk.ipynb

https://www.tutorialspoint.com/gensim/gensim_documents_and_lda_model.htm

https://www.tutorialspoint.com/gensim/gensim_creating_lda_topic_model.htm

# Further Package Installations

In [ ]:
#Delete?
#from __future__ import unicode_literals, print_function
#import re

import matplotlib.pyplot as plt
%matplotlib inline
from importlib import reload
from pprint import pprint
#import sys
get_ipython().run_line_magic('matplotlib', 'inline')
from __future__ import unicode_literals

In [ ]:
#Delete?
# Enable logging for Gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# Set up log to external log file
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
## Importing Packages
# source - https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# Set up log to external log file
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
## Importing Data

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alwer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from __future__ import unicode_literals
import spacy
##spacy.load('en')
#from spacy.lang.en import English
#parser = English()

In [ ]:
#Import my texts
import pandas as pd
import numpy as np

import sys
from importlib import reload
reload(sys)
stdout = sys.stdout
reload(sys)


<module 'sys' (built-in)>

In [ ]:
#sys.setdefaultencoding('utf-8')
sys.stdout = stdout
#encoding = 'unicode_escape'

#df = pd.read_csv('nova.csv', encoding="unicode_escape")
# "utf-8"   names = ['review'],    

#import csv
#df = pd.read_excel('video.csv') 
#print (df)

FileNotFoundError: [Errno 2] No such file or directory: 'video.csv'

# Import Documents into Corpus

In [ ]:
# Import Documents into Corpus
import glob
import os
file_list = glob.glob(os.path.join(os.getcwd(),"C:\\Users\\alwer\\Desktop\\Coding Working Group\\test", "*.txt"))

corpus = []

for file_path in file_list:
    with open(file_path, encoding = "utf8", errors='ignore') as f_input:
        corpus.append(f_input.read())
        
print(corpus[:2])  

# Change File List to Title - to work on
my_dir = "C:\\Users\\alwer\\Desktop\\Coding Working Group\\test"
filelist = []
filesList = []
os.chdir( my_dir )

# Step 2: Build up list of files:
for files in glob.glob("*.txt"):
    fileName, fileExtension = os.path.splitext(files)
    filelist.append(fileName) #filename without extension
    
# Import Documents into Dataframe
corpus = np.array(corpus)
corpus_df = pd.DataFrame({'title': filelist, 'text': corpus})
df = corpus_df[['title', 'text']]
df

In [ ]:
print(df.text.unique())
df.head()  

# Text Cleaning

In [ ]:
# Convert to list
data = df.text.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
from __future__ import unicode_literals

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
# data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en

#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
#data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized[:1])


# Visualize Corpora

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_nostops)
#id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
#texts = data_lemmatized
texts = data_words_nostops
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 3), (2, 1), (3, 1), (4, 8), (5, 1), (6, 8), (7, 1), (8, 1), (9, 9), (10, 3), (11, 1), (12, 2), (13, 4), (14, 6), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 3), (22, 3), (23, 1), (24, 1), (25, 4), (26, 1), (27, 4), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 2), (34, 2), (35, 2), (36, 2), (37, 2), (38, 4), (39, 1), (40, 3), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 40), (52, 7), (53, 6), (54, 1), (55, 1), (56, 4), (57, 2), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 21), (68, 1), (69, 2), (70, 1), (71, 1), (72, 9), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 3), (82, 2), (83, 1), (84, 5), (85, 1), (86, 3), (87, 2), (88, 1), (89, 1), (90, 1), (91, 3), (92, 1), (93, 4), (94, 1), (95, 4), (96, 1), (97, 1), (98, 2), (99, 1), (100, 1), (101, 1), (102, 2), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 11), (109, 1), (110,

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

# Refine Topic Model

In [ ]:
# print for 3
from __future__ import print_function

# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
from __future__ import print_function

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_nostops, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
with open('coherence.txt', 'w') as f:
    print("perplexity =", lda_model.log_perplexity(corpus), "coherence =", coherence_lda, file=f)

# Further Experiments

In [ ]:
#Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = 'mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt


# Show graph
limit=160; start=2; step=24;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))
    


In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

# Script from Older File, not sure if it goes in this order

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

dominant_topic = df_dominant_topic.head(20)
with open('dominanttopic.csv', 'w') as f:
    print(dominant_topic, file=f)

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]


In [ ]:
# Show
sent_topics_sorteddf_mallet.head(30)


In [ ]:
sorted_topics = sent_topics_sorteddf_mallet.head(30)
with open('phrases.csv', 'w') as f:
    print(sorted_topics, file=f)

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()


# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

# Another section, maybe out of order?

In [ ]:
import re
pos_re = re.compile(r'/(NOUN|ADJ|VERB|ADV)')

In [ ]:

def extract_dists(model, sf='text'):
    data = pyLDAvis.graphlab._extract_data(model, sf['bow'])
    vocab = data['vocab'] = [pos_re.sub('', t).replace('_', ' ') for t in data['vocab']]
    vis_data = pyLDAvis.prepare(**data)
    vis_topic_order = vis_data.topic_order
    new_order = np.array(vis_topic_order) - 1
    topic_ids = range(1, len(new_order) + 1)    
    data['topic_term_dists'] = pd.DataFrame(data['topic_term_dists'].T, index=vocab)[new_order]
    data['topic_term_dists'].columns = topic_ids
    data['doc_topic_dists'] = pd.DataFrame(data['doc_topic_dists'], index=sf['title'])[new_order]
    data['doc_topic_dists'].columns = topic_ids
    if vis_data:
        data['vis'] = vis_data
    return data

In [ ]:
model_data = extract_dists(doc_lda)

def topics_for(doc_name, doc_dist=model_data['doc_topic_dists']):
    return doc_dist.ix[doc_name].order(ascending=False)

def _sort_cols(df, cols):
    res = df[cols].apply(lambda probs: probs.order(ascending=False).index)
    return res.reset_index(drop=True)

def top_topic_terms(topic_ids, topic_term_dists=model_data['topic_term_dists']):
    return _sort_cols(topic_term_dists, topic_ids)

def top_docs(topic_ids, doc_topic_dists=model_data['doc_topic_dists']):
    return _sort_cols(doc_topic_dists, topic_ids)

def top_term_topics(term, topic_term_dists=model_data['topic_term_dists']):
    df = topic_term_dists.T[term].order(ascending=False)
    return df#.reset_index(drop=True)

def all_top_terms(topic_term_dists=model_data['topic_term_dists']):
    return top_topic_terms(topic_term_dists.columns)

def topic_docs(topic_id, doc_topic_dists=model_data['doc_topic_dists']):
    return doc_topic_dists[topic_id].order(ascending=False)

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

dominant_topic = df_dominant_topic.head(20)
with open('dominanttopic.csv', 'w') as f:
    print(dominant_topic, file=f)

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]


In [ ]:
# Show
sent_topics_sorteddf_mallet.head(30)

sorted_topics = sent_topics_sorteddf_mallet.head(30)
with open('phrases.csv', 'w') as f:
    print(sorted_topics, file=f)

    

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()


# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

In [ ]:
import re
pos_re = re.compile(r'/(NOUN|ADJ|VERB|ADV)')

def extract_dists(model, sf='text'):
    data = pyLDAvis.graphlab._extract_data(model, sf['bow'])
    vocab = data['vocab'] = [pos_re.sub('', t).replace('_', ' ') for t in data['vocab']]
    vis_data = pyLDAvis.prepare(**data)
    vis_topic_order = vis_data.topic_order
    new_order = np.array(vis_topic_order) - 1
    topic_ids = range(1, len(new_order) + 1)    
    data['topic_term_dists'] = pd.DataFrame(data['topic_term_dists'].T, index=vocab)[new_order]
    data['topic_term_dists'].columns = topic_ids
    data['doc_topic_dists'] = pd.DataFrame(data['doc_topic_dists'], index=sf['title'])[new_order]
    data['doc_topic_dists'].columns = topic_ids
    if vis_data:
        data['vis'] = vis_data
    return data

In [ ]:
model_data = extract_dists(doc_lda)

def topics_for(doc_name, doc_dist=model_data['doc_topic_dists']):
    return doc_dist.ix[doc_name].order(ascending=False)

def _sort_cols(df, cols):
    res = df[cols].apply(lambda probs: probs.order(ascending=False).index)
    return res.reset_index(drop=True)

def top_topic_terms(topic_ids, topic_term_dists=model_data['topic_term_dists']):
    return _sort_cols(topic_term_dists, topic_ids)

def top_docs(topic_ids, doc_topic_dists=model_data['doc_topic_dists']):
    return _sort_cols(doc_topic_dists, topic_ids)

def top_term_topics(term, topic_term_dists=model_data['topic_term_dists']):
    df = topic_term_dists.T[term].order(ascending=False)
    return df#.reset_index(drop=True)

def all_top_terms(topic_term_dists=model_data['topic_term_dists']):
    return top_topic_terms(topic_term_dists.columns)

def topic_docs(topic_id, doc_topic_dists=model_data['doc_topic_dists']):
    return doc_topic_dists[topic_id].order(ascending=False)

In [ ]:
code_topics = top_term_topics('kids')[0:10]
code_topics

In [ ]:
code_topics = code_topics[code_topics > 0.01]
len(code_topics)

In [ ]:
top_docs(code_topics.index).head(5)

In [ ]:
code_topics = code_topics[code_topics > 0.02]

In [ ]:
docs_ordred_by_code = model_data['doc_topic_dists'][code_topics.index].sum(axis=1).order(ascending=False)
def fit_focused_model(ordered_docs, num_topics, num_iters=100, threshold=0.1):
    subset = set(ordered_docs[ordered_docs > threshold].index)
    print('Keeping %.2f%% of the corpus...' % (100 * (len(subset) / len(ordered_docs))))
    # I should have kept the doc index around, oh well..
    stories_subset = stories_sf[stories_sf['title'].apply(lambda t: t in subset)]
    bows = stories_subset['bow']    
    print('Fitting model...')
    tm = gl.topic_model.create(bows, num_topics, num_iterations=num_iters)
    print('Creating vis data...')
    data = extract_dists(tm, stories_subset)
    data['model'] = tm
    return data
code_model = fit_focused_model(docs_ordred_by_code, 40, num_iters=500, threshold=0.25)

In [ ]:
code_model['vis']

# Stop Words

In [ ]:
import nltk; nltk.download('stopwords')
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])